In [1]:
import pandas as pd

coef_df = pd.read_csv('coef.csv')
coef_df

,y,x,mean,std,p-value,FDR adj p-value
0,Female,Female,0.901184,0.011805,1.206825e-14,2.235362e-11
1,Deceased,Age 70-74,0.016640,0.004791,2.405332e-05,1.152237e-02
2,Age 0-4,Age 0-4,0.732519,0.004704,8.214257e-17,7.607497e-13
3,Age 5-9,Age 0-4,0.092131,0.005375,4.155717e-10,3.608201e-07
4,Age 5-9,Age 5-9,0.692818,0.003915,3.356313e-17,4.662590e-13
...,...,...,...,...,...,...
56,Recurrent rectal polyp,Polyp of colon,0.068193,0.013384,1.845046e-06,9.858221e-04
57,Chronic pain,Chronic intractable migraine without aura,0.077430,0.013191,7.023891e-07,3.982690e-04
58,Chronic intractable migraine without aura,Impacted molars,0.065349,0.022253,7.165796e-05,3.263844e-02
59,Hyperglycemia (disorder),Diabetes,0.028905,0.008455,2.668627e-05,1.256697e-02


In [61]:
constant_df = pd.read_csv('constant.csv')
constant_df['label'] ='constant'
constant_df

,mean,std,p-value,FDR adj p-value,name,label
0,0.041036,0.009143,4.354037e-06,1.423027e-05,Female,constant
1,0.033264,0.002216,1.050355e-09,5.026697e-09,Deceased,constant
2,0.003362,0.001561,4.955131e-04,1.106646e-03,Age 0-4,constant
3,0.009585,0.002250,6.196918e-06,1.931133e-05,Age 5-9,constant
4,0.009626,0.002633,1.713182e-05,4.631115e-05,Age 10-14,constant
...,...,...,...,...,...,...
129,0.000159,0.000591,4.725763e-01,5.412412e-01,"Localized, primary osteoarthritis of the hand",constant
130,-0.000319,0.000368,4.406098e-02,7.200209e-02,Osteoarthritis of hip,constant
131,0.000112,0.000687,6.598739e-01,7.073848e-01,Contact dermatitis,constant
132,0.000023,0.000838,9.390528e-01,9.605579e-01,Concussion injury of brain,constant


In [47]:
#server = 'http://67.205.191.70'
server = 'https://yaniv.i.tgcloud.io'

import getpass

password = getpass.getpass()

········


In [9]:

!pip install pyTigerGraph

In [48]:
import pyTigerGraph as tg 

# Replace graphname='social' with the graph you would like to be used as default
conn = tg.TigerGraphConnection(
    host=server, 
    graphname='ML', 
    password=password)

# Change client_version once TigerGraph Cloud is upgraded  
#shell = tg.Gsql(conn, certNeeded=False)
shell = tg.Gsql(conn, certNeeded=True)

In [15]:
print(shell.gsql('ls', options=[]))

Trying version: v3_0_0
Invalid port 14240/. Falling back to default port 14240.
Connecting to 67.205.191.70:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
  - VERTEX _Gender(PRIMARY_ID gender_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX _Race(PRIMARY_ID race_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX _Ethnicity(PRIMARY_ID ethnicity_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX _Address(PRIMARY_ID address_id STRING, name STRING, lat DOUBLE, lon DOUBLE) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX _State(PRIMARY_ID state_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  - VERTEX _County(PRIMARY_ID county_id STRING, countyName STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"


In [43]:
print(shell.gsql('''
    drop graph ML
    drop edge ML_reg_constant
    drop edge ML_reg_coef
    drop vertex ML_condition
    drop vertex ML_label
''', options=[]))

Trying version: v3_0_0
Connecting to 67.205.191.70:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The graph ML is dropped.
The edge type ML_reg_constant is dropped.
The edge type ML_reg_coef is dropped.
The vertex type ML_condition is dropped.
The vertex type ML_label is dropped.



In [53]:
print(shell.gsql('''
    create vertex ML_condition(primary_id condition_id string) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
    create vertex ML_label(primary_id label_id string) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
    create undirected edge ML_reg_constant(from ML_condition, to ML_label, mean double, std double, p_value double, adj_p_value double)   
    create directed edge ML_reg_coef(from ML_condition, to ML_condition, mean double, std double, p_value double, adj_p_value double) 
    
    create graph ML (ML_condition, ML_label, ML_reg_constant, ML_reg_coef)
''', options=[]))

Trying version: v3_0_0
Connecting to yaniv.i.tgcloud.io:14240
GSQL client is not compatible with GSQL server. Please contact support@tigergraph.com
Supported Versions ( v3_0_0 v2_6_0 v2_5_2 v2_5_0 v2_4_1 v2_4_0 v2_3_2 )
You may use 'GSQL_CLIENT_VERSION=v? java ...' or 
    'java -DGSQL_CLIENT_VERSION=v? ...' to specify the version
Trying version: v2_6_0
Connecting to yaniv.i.tgcloud.io:14240
If there is any relative path, it is relative to tigergraph/dev/gdk/gsql
The vertex type ML_condition is created.
The vertex type ML_label is created.
The edge type ML_reg_constant is created.
The edge type ML_reg_coef is created.
The graph ML is created.



In [55]:
secret = shell.createSecret()
len(secret)

32

In [57]:
token = conn.getToken(secret=secret)
len(token[0])

32

In [58]:
conn.upsertVertex(vertexType='ML_label', vertexId='constant', attributes={'label_id': 'constant'})

1

In [59]:
conn.upsertVertexDataframe(df=constant_df, vertexType='ML_condition', v_id='name', attributes={'condition_id':'name'})

134

In [62]:
conn.upsertEdgesDataframe(
    df=constant_df, 
    sourceVertexType='ML_condition', 
    edgeType='ML_reg_constant', 
    targetVertexType='ML_label', 
    from_id='name', 
    to_id='label', 
    attributes={'mean':'mean', 'std':'std', 'p_value':'p-value', 'adj_p_value': 'FDR adj p-value'},
)


134

In [63]:
conn.upsertEdgesDataframe(
    df=coef_df, 
    sourceVertexType='ML_condition', 
    edgeType='ML_reg_coef', 
    targetVertexType='ML_condition', 
    from_id='x', 
    to_id='y', 
    attributes={'mean':'mean', 'std':'std', 'p_value':'p-value', 'adj_p_value': 'FDR adj p-value'},
)

61